**Draft Model for Mice Steep Stage Analysis**

In [1]:
import random
import numpy as np
import pandas as pd
import os
import re
import copy
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
from torchvision import transforms, datasets, models

from collections import OrderedDict

import torch.utils.data as utils

from sklearn.preprocessing import LabelBinarizer

import tkinter as tk #tk for file dialog (requires Jinja2!!!)
from tkinter import filedialog #tkinter for file dialog

import re #regex for parsing
from os.path import exists


In [6]:
from inputMassager import *
inputHandler = inputMassager()

filepath = inputHandler.askForInput("Blah!")

#Period Size Variable : effects CNN architecture
periodSize = 200

#makePeriodFromTxt(self, filepath, periodSize, maxPeriods=None):
periods = inputHandler.makePeriodFromTxt(filepath, periodSize, 2000)


# target = regan and audreys code ---> This is what will go into the data loader
filepath2 = inputHandler.askForInput("Blah!")
timestamps = find_time_labels(df, filepath2)
labels = label_dataframe(periods, timestamps)


TclError: ignored

Make a tensor out of eeg data (c1)

In [ ]:
def getC1Tensor():
    #The tensor has to be 3 dimesional with the second dimesion equal to one in order to pass it through a 1d convolutional layer
    c1_tensor = torch.zeros((len(periods.c1.values), 1, 200)) 

    for i in range(len(periods.c1.values)):
        c1_tensor[i, 0] = torch.tensor(periods.c1.values[i])
    
    return c1_tensor

eeg_samples = getC1Tensor()

Make the train and validation data loaders  

In [ ]:
ds = torch.utils.data.TensorDataset(eeg_samples, torch.tensor(labels))
train_dataset, val_dataset = torch.utils.data.random_split(ds, [int(len(ds) *.80), int(len(ds) *.2)])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Creating class for Model**

In [ ]:
import torch.nn.functional as F
from torch import nn, optim


class Model_3(nn.Module):
    def __init__(self, n_feature, output_size):
        super(Model_3, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv1d(in_channels=1, out_channels= 100, kernel_size=2)
        #out_channels and kernal size are random
        self.fc1 = nn.Linear(9900, 10)#10 is random


    def forward(self, x, verbose=False):
        #print(x.size())
        x = self.conv1(x)
        x = F.relu(x)
        #print(x.size())
        x = F.max_pool1d(x, kernel_size=2)
        #print(x.size())
        x = x.flatten(1)
        #print(x.size())
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x

In [2]:
def train_model(epochs, model):
    model.train()#set model to training mode
    loss_fun = nn.CrossEntropyLoss()#define a loss function object

    for epoch in range(epochs):

      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device), target.to(device)
          
          optimizer.zero_grad()
          output = model(data)
          loss = loss_fun(output,target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
model_step_3 = Model_3(200, 32)
model_step_3.to(device)
  
optimizer = torch.optim.SGD(model_step_3.parameters(), lr=.01)
train_model(3, model_step_3)

In [ ]:
def evaluate_model(model, dataloader, is_test=False):
  #Evaluation

  # Set model to evaluation mode
  model.eval()

  with torch.no_grad():
    correct = 0
    loss = 0

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        
        loss += torch.sum(criterion(outputs, target)).item()
        
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        "Test" if is_test else "Validation",
        loss, correct, len(dataloader.dataset),
        accuracy))
  # Set model back to training mode
  model.train()

In [ ]:
evaluate_model(model_step_3, val_loader, is_test=True)